In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook

import cv2
from skimage import io
from skimage.color import rgb2gray
from skimage import exposure as ep

from skimage import feature as ft
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_dir = 'data/train/'
validation_dir = 'data/validation/'
test_dir = 'data/test/'


output_classes = 2
batch_size = 32 
img_height, img_width = 256, 256

# nb_train_samples = 1188
# nb_validation_samples = 144
# nb_test_samples = 144

In [4]:
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['normal']:
                label = 0
            elif folderName in ['malignant']:
                label = 1
#             else:
#                 label = 3
            for image_filename in tqdm_notebook(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
#                     img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [5]:
X_train, y_train = get_data(train_dir)

In [6]:
X_validation, y_validation = get_data(validation_dir)

In [7]:
X_test, y_test = get_data(test_dir)

In [8]:
from skimage import feature
import numpy as np

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist

# settings for LBP
radius = 3
n_points = 8 * radius
desc = LocalBinaryPatterns(24, 8)

In [9]:
trainSetList = []
validationSetList = []
testSetList = []

In [10]:
for imgs in X_train:
    gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    trainSetList.append(hist.ravel())
    pass

In [11]:
for imgs in X_validation:
    gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    validationSetList.append(hist.ravel())
    pass

In [12]:
for imgs in X_test:
    gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    testSetList.append(hist.ravel())
    pass

trainSet = np.array(trainSetList, np.float32)
validationSet = np.array(validationSetList, np.float32)
testSet = np.array(testSetList, np.float32)

In [13]:
# from keras.utils.np_utils import to_categorical
# y_trainHot = to_categorical(y_train, num_classes = 2)
# y_valHot = to_categorical(y_validation, num_classes = 2)
# y_testHot = to_categorical(y_test, num_classes = 2)

In [14]:
trainSet.shape

(59168, 26)

In [15]:
validationSet.shape

(18384, 26)

In [16]:
testSet.shape

(15472, 26)

In [17]:
y_train.shape

(59168,)

In [18]:
# y_trainHot.shape

In [19]:
y_test.shape

(15472,)

In [20]:
y_test

array([1, 1, 1, ..., 0, 0, 0])

In [25]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]

model = RandomForestClassifier(n_estimators=100)

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result )

    CV Mean       Std
0  0.945121  0.058946
folds accuracies: [0.98320413 0.98837209 0.99030381 0.98771816 0.98254686 0.95604396
 0.9844861  0.81965094 0.88752424 0.87136393]
